This code uses built-in functions from the Image class in PIL to execute isometries on a given image. It snapshots the lens view after each isometry, then puts those images into a numpy array. It then calculates the persistent homology of the data. For different images, 'lens_sz' and 'intervals' should be ajusted.

In [32]:
# Imports
import numpy as np
import math

# For image and video manipulation
from PIL import Image
from PIL import ImageEnhance
import glob

# Persistent homology
from ripser import ripser
from persim import plot_diagrams

import matplotlib.pyplot as plt
%matplotlib inline

In [38]:
# Grab image from files
img_names = glob.glob('images/*C1arrows.png')
img = Image.open(img_names[0])
img = ImageEnhance.Sharpness(img)
img = img.enhance(0.5)

img.show()

translation_snapshot() translates the plane by a vector a and returns the lens view afterwards. It takes an image (Image), the lens size (int), and the translation vector (tuple) that shifts the plane in the direction of the vector. 

In [17]:
def translation_snapshot(img, lens_sz, a):
    (x,y)=a
    x=-x #since x and y will shift the crop and not the plane, need to make sure it is going in right direction
    (length, height) = img.size
    lens_corner_x=length/2.0 - lens_sz/2.0 +x
    lens_corner_y=height/2.0 - lens_sz/2.0 +y
    ret_img = img.crop((lens_corner_x,lens_corner_y, lens_corner_x+lens_sz, lens_corner_y+lens_sz))
    
    #make sure didn't fall off of the image
    (ret_length, ret_height) = ret_img.size
    if (lens_corner_x<0 or lens_corner_y<0 or lens_corner_x+lens_sz>length or lens_corner_y+lens_sz>height):
        print("I came off the page!")
        return False
    return ret_img

In [27]:
# adjustable variables
lens_sz = 48
degree_num = 70
translation_num = 5

image_list = []

# this range depends on image size and lens size such that the lens doesn't come off the page
lens_range = ((min(img.size)/math.sqrt(2))-lens_sz)/2.0

degrees = np.linspace(0, 360, degree_num)
intervals_x = np.linspace((-1)*lens_range, lens_range, translation_num)
intervals_y = np.linspace((-1)*lens_range, lens_range, translation_num)

# take snapshot images
for d in degrees:
    rot = img.rotate(d)
    for x in intervals_x:
        for y in intervals_y:
            image_list.append(translation_snapshot(rot, lens_sz, (x,y)))
    
# Turn images into array of data
data = []
x=0
for i in image_list:
    img_arr = np.asarray(i)
    data.append(img_arr.reshape(-1))
data = np.array(data)


lens size:  48
image size:  (1080, 662)
lens range:  210.05234457274722


/Users/shannagoldman/anaconda3/envs/dreimac/lib/python3.8/site-packages/ripser/ripser.py:253: UserWarning: The input point cloud has more columns than rows; did you mean to transpose?
  warnings.warn(


In [28]:
# make persistent homology plot with ripser
res = ripser(data, maxdim=1,  coeff =2)
dgms = res['dgms']
plt.figure(figsize = (4,4)) 
plot_diagrams(dgms)

In [14]:
# if you want to store generated images in a file
def store_snapshots(image_ls):
    counter = 0
    for img1 in image_ls:
        img1.save("images/dataset/"+str(counter)+".png")
        counter+=1

In [26]:
# see how many points are persisting
H1 = res['dgms'][1].tolist() # all intervals in H1
d = []
for pair in H1: 
    d.append(pair[1] - pair[0]) # finding lengths of intervals in H1
d.sort(reverse = True) # sorting from longest to shortest intervals
d

[973.1357421875,
 969.92822265625,
 869.281982421875,
 644.696533203125,
 637.419189453125,
 634.27734375,
 515.952880859375,
 515.816650390625,
 504.587158203125,
 485.969482421875,
 481.878173828125,
 481.878173828125,
 481.22412109375,
 481.22412109375,
 449.7294921875,
 448.425537109375,
 434.189208984375,
 431.2236328125,
 423.101318359375,
 421.105712890625,
 413.8955078125,
 408.51416015625,
 406.605224609375,
 406.087890625,
 405.976806640625,
 404.9375,
 401.5498046875,
 400.748779296875,
 398.3271484375,
 397.471923828125,
 396.24755859375,
 395.994873046875,
 386.7275390625,
 386.1337890625,
 382.636474609375,
 377.2763671875,
 375.355712890625,
 374.951171875,
 372.95068359375,
 372.41064453125,
 371.870361328125,
 371.7802734375,
 371.389404296875,
 370.03662109375,
 369.1474609375,
 368.566650390625,
 368.1123046875,
 367.414306640625,
 367.033447265625,
 365.859375,
 363.72265625,
 361.460205078125,
 359.937255859375,
 358.630615234375,
 355.543212890625,
 352.5112304687